# Get Renewable Ninja Data

**Objective**
Download, store, and explore Renewable Ninja wind and solar generation profiles for representative locations that will feed the EPM capacity-expansion model.

**Data Requirements & Methods**
- A Renewable Ninja account/API token plus the helper functions defined in `utils_renewables.py`.
- User-provided coordinates per country (see `get_renewables_coordinate.ipynb`) and a writable `input/` + `output/` folder.
- The notebook orchestrates API pulls (or reads cached CSVs), organizes the files, and produces quick exploration plots via pandas/matplotlib/seaborn.

**Overview of Steps**
1. Step 1 - Configure user parameters and import helper utilities.
2. Step 2 - Build the expected folder structure for raw and processed data.
3. Step 3 - Optionally query Renewable Ninja for hourly profiles.
4. Step 4 - Explore, format, and visualize the resulting datasets.



In [1]:
import os
from utils_renewables import *

## Step 1 - Configure user parameters and import helpers
Edit the scenario name, API extraction flags, coordinates, technologies, and time horizon before running the rest of the notebook.



In [6]:
name_data = 'capp' # Name of the zone, used to name the folder where the data is stored

extract_renewables = {'solar': True, 'wind': True} # Set to True to extract data from Renewables Ninja

# If extract_renewables is True, fill the following variables
# Best is to use the same zone name than the one in EPM
# The coordinates can be found thanks to the get_renewables_coordinates.ipynb notebook
locations = {'solar': {'Angola': (-12.4667, 15.7442),
                       #'Burundi': (-3.1479, 29.3625),
                       'Cameroon': (9.3271, 13.3621),
                       #'CAR': (4.5167, 18.4667),
                       #'Chad': (12.0905, 15.1331),
                       'Congo': (1.3863, 16.3364),
                       'DRC': (-1.6414, 29.1938),
                       'Gabon': (0.2293, 9.5315),
                       #'EquatorialGuinea': (1.6132, 10.6400),
                       },
             'wind': {'Angola': (-15.2198, 12.7258),
                      'Cameroon': (4.8918, 12.3598),
                      'Chad': (12.1232, 15.0712)}}


start_year = 2023 # earliest year is 1980
end_year = 2024


## Step 2 - Create the folder structure
Ensure `input/` stores raw Renewable Ninja payloads and `output/` stores diagnostics and figures.



In [7]:
folder_input = 'input'
# Make folder
if not os.path.exists(folder_input):
    os.makedirs(folder_input)
    
folder_output = 'output'
if not os.path.exists(folder_output):
    os.makedirs(folder_output)

power_types = list(locations.keys()) # 'wind' or 'solar'

print(f'Extracting data from {start_year} to {end_year} for {name_data}')

Extracting data from 2023 to 2024 for capp


## Step 3 - Extract data from Renewable Ninja
Loop over the requested technologies and locations, calling the API only when `extract_renewables` flags are set to True.



In [8]:
for power_type in power_types:
    print('Processing power type {}'.format(power_type))
    if extract_renewables[power_type]:
        locations_power = locations[power_type]
        get_years_renewables(locations=locations_power, power_type=power_type,
                             start_year=start_year, end_year=end_year,
                             name_data=name_data, output=folder_input,
                             local_time=True)

Processing power type solar
Fetching data for: Angola
Fetching data for: Cameroon
Fetching data for: Congo
Fetching data for: DRC
Fetching data for: Gabon
Getting data solar for year 2023
Export data to input/data_capp_solar.csv
Processing power type wind
Fetching data for: Angola
Fetching data for: Cameroon
Fetching data for: Chad
Getting data wind for year 2023
Export data to input/data_capp_wind.csv


## Step 4 - Explore and format data
Load the cached CSV files, assemble tidy DataFrames, and run quick plots (heatmaps and boxplots) to QC the profiles before downstream use.



In [9]:
filenames = {'PV': os.path.join(folder_input, 'data_{}_solar.csv'.format(name_data)),
             'Wind': os.path.join(folder_input, 'data_{}_wind.csv'.format(name_data)),
             }

df_energy = {}
for tech, filename in filenames.items():
    if os.path.exists(filename):
        print(f'Loading data from {filename}')
        df_energy[tech] = pd.read_csv(filename, index_col=None, header=[0])
    else:
        print(f'File {filename} does not exist. Please check the input folder.')

# Make heatmaps and boxplots to explore the data
for tech, df in df_energy.items():
    make_heatmap(df, 
                 tech=tech, 
                 # path=os.path.join(folder_output, f'heatmap_{tech}_{name_data}.pdf')
                 )
    make_boxplot(df, 
                 tech=tech,
                 #path=os.path.join(folder_output, f'boxplot_{tech}_{name_data}.pdf'
                 )

Loading data from input/data_capp_solar.csv
Loading data from input/data_capp_wind.csv


ValueError: could not convert string to float: '1 - 1'

<Figure size 1200x600 with 0 Axes>